In [427]:
from math import inf
from os import path, sep
from collections import defaultdict
import pandas as pd
import numpy as np
import pdb
import re

In [6]:
df = pd.read_csv('input.txt', header=None)

In [8]:
df.columns = ['command']

In [73]:
df

,command
0,$ cd /
1,$ ls
2,dir dfmhjhd
3,307728 ghpqs
4,dir hztjntff
...,...
1042,$ cd ..
1043,$ cd wrmm
1044,$ ls
1045,212937 frcqrdm


In [12]:
nav_df = df[df['command'].str.startswith('$ cd')]

In [21]:
nav_df[nav_df['command'].str.contains('\.\.')]

,command
13,$ cd ..
31,$ cd ..
32,$ cd ..
36,$ cd ..
37,$ cd ..
...,...
1031,$ cd ..
1036,$ cd ..
1040,$ cd ..
1041,$ cd ..


In [22]:
nav_df

,command
0,$ cd /
9,$ cd dfmhjhd
13,$ cd ..
14,$ cd hztjntff
22,$ cd cwsf
...,...
1037,$ cd vdqmtjw
1040,$ cd ..
1041,$ cd ..
1042,$ cd ..


In [43]:
class folder:
    def __init__(self, folder_name, parent_folder=None):
        self.parent_folder = parent_folder
        self.subfolders = {}
        self.files = {}
        self.folder_name = folder_name
    

In [50]:
nav_df.iloc[1:]

,command
9,$ cd dfmhjhd
13,$ cd ..
14,$ cd hztjntff
22,$ cd cwsf
28,$ cd msr
...,...
1037,$ cd vdqmtjw
1040,$ cd ..
1041,$ cd ..
1042,$ cd ..


In [142]:
current_directory_level = 0
fs = folder(folder_name = 'root', parent_folder=None)
current_directory = fs

all_folders = []

for enum, row in df.iloc[1:].iterrows():
    if (row['command'] == '$ ls') & (row['command'].startswith('dir ')):
        continue
    folder_name = re.search('cd (\w+|..)', row['command'])
    if folder_name:
        folder_name = folder_name.groups()[0]
        
        if folder_name != '..':
            current_directory.subfolders[folder_name] = folder(folder_name, parent_folder=current_directory)
            current_directory = current_directory.subfolders[folder_name]
            current_directory_level = current_directory_level+1
            all_folders.append((current_directory.folder_name,current_directory, current_directory_level))
            continue
        else:
            current_directory = current_directory.parent_folder
            current_directory_level = current_directory_level-1
            continue
    file_info = re.search('(\d+) ', row['command'])
    if file_info:
        current_directory.files[row['command'].split(' ')[1]]=int(row['command'].split(' ')[0])

In [145]:
all_folders = pd.DataFrame(all_folders)

In [147]:
all_folders.columns = ['folder_name', 'folder_obj', 'depth']

In [155]:
has_children = []
for enum, row in all_folders.iterrows():
    if len(row['folder_obj'].subfolders) == 0:
        has_children.append(False)
    else:
        has_children.append(True)

In [156]:
all_folders['has_children'] = has_children

In [159]:
all_folders

,folder_name,folder_obj,depth,has_children
0,dfmhjhd,<__main__.folder object at 0x2aaaf8544f70>,1,False
1,hztjntff,<__main__.folder object at 0x2aaaf7ed9dc0>,1,True
2,cwsf,<__main__.folder object at 0x2aaaf7043fa0>,2,True
3,msr,<__main__.folder object at 0x2aaaf7043c70>,3,False
4,sthqhrc,<__main__.folder object at 0x2aaaf7043670>,2,False
...,...,...,...,...
182,zmtmr,<__main__.folder object at 0x2aaaf8c6cf10>,2,True
183,sms,<__main__.folder object at 0x2aaaf8c6cf70>,3,False
184,vdn,<__main__.folder object at 0x2aaaf8c6ce20>,3,False
185,vdqmtjw,<__main__.folder object at 0x2aaaf8c6cd90>,3,False


In [420]:
def addUpFiles(folder):
    if len(folder.files) > 0:
        return(np.sum(list(folder.files.values())))
    else:
        return(int(0))
def traverse(folder, running_total):
    #print(folder.folder_name)
    for folder_name, foldo in folder.subfolders.items():
        if len(foldo.subfolders.items()) == 0:
            running_total = running_total + addUpFiles(foldo)
        else:
            running_total = traverse(foldo, running_total)
    return(running_total)


In [421]:
recursive_folder_size = []
for enum, row in all_folders.iterrows():
    if row['has_children'] == False:
        recursive_folder_size.append(addUpFiles(row['folder_obj']))
    else:
        running_total = addUpFiles(row['folder_obj'])
        running_total = traverse(row['folder_obj'], running_total)
        recursive_folder_size.append(running_total)
all_folders['recursive_folder_size'] = recursive_folder_size

In [422]:
all_folders[all_folders['recursive_folder_size'] < 100000]['recursive_folder_size'].sum()

1824177

In [318]:
283543 + 327156 + 291828 +285763


1188290

In [319]:
addUpFiles(all_folders.iloc[182]['folder_obj'])

285763

In [320]:
fs.subfolders['fmhjh']

KeyError: 'fmhjh'

In [ ]:
p1 = 0
p2 = inf
cwd = ""
dir_sizes = defaultdict(int)

with open("./input.txt") as f:
    for line in f:
        parts = line.strip().split(" ")

        if parts[0] == "$" and parts[1] == "cd":
            cwd = path.normpath(path.join(cwd, parts[2]))

        if parts[0].isnumeric():
            dirs = cwd.split(sep)
            for i in range(len(dirs)):
                pdb.set_trace()
                dir_path = path.normpath(path.join(*dirs[: i + 1]))
                dir_sizes[dir_path] += int(parts[0])

avail_space = 7e7 - dir_sizes["."]
sizes = dir_sizes.values()

p1 = sum((v for v in sizes if v <= 1e5))
p2 = min((v for v in sizes if v + avail_space >= 3e7))

print(f"p1: {p1}, p2: {p2}")



> /scratch/tmp/22797403.1.default.q/ipykernel_26346/4144310828.py(17)<cell line: 6>()
     15             for i in range(len(dirs)):
     16                 pdb.set_trace()
---> 17                 dir_path = path.normpath(path.join(*dirs[: i + 1]))
     18                 dir_sizes[dir_path] += int(parts[0])
     19 



ipdb>  dirs


['', '']


ipdb>  n


> /scratch/tmp/22797403.1.default.q/ipykernel_26346/4144310828.py(18)<cell line: 6>()
     16                 pdb.set_trace()
     17                 dir_path = path.normpath(path.join(*dirs[: i + 1]))
---> 18                 dir_sizes[dir_path] += int(parts[0])
     19 
     20 avail_space = 7e7 - dir_sizes["."]



...

['301486', 'ngtqtf']


ipdb>  dirs


['', 'dfmhjhd']


ipdb>  parts[0]


'301486'


In [430]:
dir_sizes

defaultdict(int,
            {'.': 47870706,
             'dfmhjhd': 314974,
             'hztjntff': 1926974,
             'hztjntff/cwsf': 1023208,
             'hztjntff/cwsf/msr': 272198,
             'hztjntff/sthqhrc': 45847,
             'rvstq': 24509789,
             'rvstq/nzdgz': 4488747,
             'rvstq/nzdgz/fplqm': 961527,
             'rvstq/nzdgz/fplqm/cwsf': 330050,
             'rvstq/nzdgz/fplqm/cwsf/fld': 330050,
             'rvstq/nzdgz/fplqm/cwsf/fld/jdpz': 299969,
             'rvstq/nzdgz/fplqm/cwsf/fld/trjltfq': 30081,
             'rvstq/nzdgz/fplqm/cwzstq': 305925,
             'rvstq/nzdgz/hzpqfj': 1179441,
             'rvstq/nzdgz/hzpqfj/sztz': 430383,
             'rvstq/nzdgz/hzpqfj/sztz/mjmbld': 101870,
             'rvstq/nzdgz/hzpqfj/sztz/nzdgz': 86208,
             'rvstq/nzdgz/hzpqfj/sztz/wrmm': 242305,
             'rvstq/nzdgz/hztjntff': 952301,
             'rvstq/nzdgz/hztjntff/hztjntff': 318839,
             'rvstq/nzdgz/hztjntff/hztjntff/

In [121]:
addUpFiles(fs)

428307

In [139]:
traverse(fs.subfolders['hztjntff'])

hztjntff
cwsf
msr


0

In [114]:
for curdir,fol in fs.subfolders.items():
    for fol.subfolder.items()
    

In [107]:
fs.subfolders

{'dfmhjhd': <__main__.folder at 0x2aaaf851c850>,
 'hztjntff': <__main__.folder at 0x2aaaf851c8e0>,
 'rvstq': <__main__.folder at 0x2aaaf851c790>,
 'sjt': <__main__.folder at 0x2aaaf8505760>,
 'wrmm': <__main__.folder at 0x2aaaf954b1c0>}